In [39]:
from PIL import Image
import numpy as np
import time

In [40]:
def local_adaptive_thresholding(image, window_size, bias):
    """
    Performs local adaptive thresholding on the input image.

    Parameters:
        image (Image.Image): Grayscale input image (Pillow Image object).
        window_size (int): Size of the local window.
        bias (float): Bias parameter for threshold adjustment.

    Returns:
        Image.Image: Thresholded binary image as a Pillow Image object.
    """
    image_array = np.array(image)
    height, width = image_array.shape

    # Create the integral image
    integral_image = np.cumsum(np.cumsum(image_array, axis=0), axis=1)

    # Initialize the thresholded image
    thresholded_image = np.zeros_like(image_array, dtype=np.uint8)

    # Half window size
    half_window = window_size // 2

    for y in range(height):
        for x in range(width):
            # Determine the coordinates of the local window
            top_left_x = max(0, x - half_window)
            top_left_y = max(0, y - half_window)
            bottom_right_x = min(width - 1, x + half_window)
            bottom_right_y = min(height - 1, y + half_window)

            # Compute the local sum using the integral image
            local_sum = integral_image[bottom_right_y, bottom_right_x]

            if top_left_y > 0:
                local_sum -= integral_image[top_left_y - 1, bottom_right_x]
            if top_left_x > 0:
                local_sum -= integral_image[bottom_right_y, top_left_x - 1]
            if top_left_x > 0 and top_left_y > 0:
                local_sum += integral_image[top_left_y - 1, top_left_x - 1]

            # Calculate the local mean
            num_pixels = (bottom_right_x - top_left_x + 1) * (bottom_right_y - top_left_y + 1)
            local_mean = local_sum / num_pixels

            # Compute the threshold
            current_pixel = image_array[y, x]
            mean_deviation = current_pixel - local_mean
            threshold = local_mean * (1 + bias * ((mean_deviation / (1 - mean_deviation)) - 1))

            # Apply the threshold
            thresholded_image[y, x] = 0 if current_pixel <= threshold else 255

    return Image.fromarray(thresholded_image)

In [41]:
# Specify parameters
w = 50
k = 0.12

# Select sample from dataset
sample = "01"

input_image = Image.open(f"Dataset/Sample{sample}.png").convert("L")

start_time = time.time()
output = local_adaptive_thresholding(input_image, w, k)
end_time = time.time()

computation_time = end_time - start_time
comp_per_pixel = computation_time / (input_image.width * input_image.height)

print(f"Tempo di esecuzione: {computation_time:.6f} secondi")
print("Dimensioni:", output.size)
print("Modalità:", output.mode)

# output.show()

Tempo di esecuzione: 0.111756 secondi


/var/folders/lj/v6tg1m8x7yxdhfvxfm2kn0000000gn/T/ipykernel_45205/653722541.py:39: RuntimeWarning: overflow encountered in scalar subtract
  local_sum -= integral_image[bottom_right_y, top_left_x - 1]
/var/folders/lj/v6tg1m8x7yxdhfvxfm2kn0000000gn/T/ipykernel_45205/653722541.py:41: RuntimeWarning: overflow encountered in scalar add
  local_sum += integral_image[top_left_y - 1, top_left_x - 1]


In [44]:
ouput_path = f"Output/Local/Output{sample}.png"
output.save(ouput_path)

with open(f"Output/Local/Output{sample}_data.txt", "w") as file:
    file.write(f"Dimensioni: {output.size} \n")
    file.write(f"Window size: {w}")
    file.write(f"Bias: {k}")
    file.write(f"Tempo di esecuzione: {computation_time:.6f} secondi\n")
    file.write(f"Tempo di esecuzione per pixel: {comp_per_pixel:.12f} secondi\n")
